In [1]:
import psycopg2
import tweepy as tw
import re
from textblob import TextBlob

In [2]:
consumer_key= ''
consumer_key_secret= ''
access_token= ''
access_token_secret= ''

auth = tw.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [4]:
search_word = "#Sinopharm+sinopharm"
tweets = tw.Cursor(api.search_tweets, q= search_word+" -filter:retweets", lang='en').items(241)
tweets_text = [remove_url(tweet.text) for tweet in tweets]

In [5]:
sentiment_objects = [TextBlob(tweet) for tweet in tweets_text]

sentiment_objects[0].polarity, sentiment_objects[0]

(0.0,
 TextBlob("Is Omicron the final VOC Article doesnt mention possible impact of inactivated virus vaccines with their ability"))

In [6]:
conn = psycopg2.connect(
    database="m-erp", user='redouan', password='redouan', host='127.0.0.1', port='5432'
)

cursor = conn.cursor()

sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

postgreSQL_Insert_Query = """INSERT INTO twitter_sinopharm (polarite, tweet) VALUES(%s, %s)"""

sentiment_values[0]

for i in sentiment_values:
    postgre_records = (i[0], i[1])
    cursor.execute(postgreSQL_Insert_Query, postgre_records)
conn.commit()

